## 1. Importing libraries

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from google.colab import drive
import cv2 as cv
from sklearn.metrics import confusion_matrix

drive.mount('gdrive')

Drive already mounted at gdrive; to attempt to forcibly remount, call drive.mount("gdrive", force_remount=True).


## 2. Importing and pre-processing data 

In [ ]:
train_ds = keras.preprocessing.image_dataset_from_directory(
  'gdrive/MyDrive/2A/MODAL/dataset4', labels='inferred', seed = 1, shuffle = True, batch_size = 64, image_size=(640, 480),color_mode='grayscale',validation_split = 0.2, subset='training')

val_ds = keras.preprocessing.image_dataset_from_directory(
  'gdrive/MyDrive/2A/MODAL/dataset4', labels='inferred', seed = 1, shuffle = True, batch_size = 64, image_size=(640, 480),color_mode='grayscale',validation_split = 0.2, subset='validation')

Found 2000 files belonging to 2 classes.
Using 1600 files for training.
Found 2000 files belonging to 2 classes.
Using 400 files for validation.


## 3. Developing the architecture of the CNN model

In [ ]:
#defining our model: inputs and outputs
from tensorflow.keras import layers

inputs = keras.Input(shape=(640, 480, 1))

# Center-crop images to 150x150
x = layers.CenterCrop(height=400, width=400)(inputs)
# Rescale images to [0, 1]
x = layers.Rescaling(scale=1.0 / 255)(x)

# Apply some convolution and pooling layers
x = layers.Conv2D(filters=32, kernel_size=(3,3), activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(3, 3))(x)
x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(3, 3))(x)
x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)

# Apply global average pooling to get flat feature vectors
x = layers.GlobalAveragePooling2D()(x)

# Add a dense classifier on top
num_classes = 2
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 640, 480, 1)]     0         
                                                                 
 center_crop_1 (CenterCrop)  (None, 400, 400, 1)       0         
                                                                 
 rescaling_1 (Rescaling)     (None, 400, 400, 1)       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 398, 398, 32)      320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 132, 132, 32)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 130, 130, 32)      9248      
                                                           

## 3. Training the model on dataset 4

In [ ]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")])

history = model.fit(train_ds, epochs=10, validation_data = val_ds)

#callbacks = tf.keras.callbacks.EarlyStopping(verbose=1, patience=3)

model.save('gdrive/MyDrive/2A/MODAL/trial8')

Epoch 1/10
25/25 [==============================] - 16s 442ms/step - loss: 0.2636 - acc: 0.9069 - val_loss: 0.2759 - val_acc: 0.8350
Epoch 2/10
25/25 [==============================] - 14s 436ms/step - loss: 0.2377 - acc: 0.9275 - val_loss: 0.2183 - val_acc: 0.9625
Epoch 3/10
25/25 [==============================] - 13s 431ms/step - loss: 0.2107 - acc: 0.9475 - val_loss: 0.1973 - val_acc: 0.9750
Epoch 4/10
25/25 [==============================] - 14s 436ms/step - loss: 0.1886 - acc: 0.9712 - val_loss: 0.1901 - val_acc: 0.9275
Epoch 5/10
25/25 [==============================] - 14s 436ms/step - loss: 0.1831 - acc: 0.9563 - val_loss: 0.1939 - val_acc: 0.9075
Epoch 6/10
25/25 [==============================] - 14s 436ms/step - loss: 0.1695 - acc: 0.9581 - val_loss: 0.1514 - val_acc: 0.9850
Epoch 7/10
25/25 [==============================] - 14s 437ms/step - loss: 0.1483 - acc: 0.9806 - val_loss: 0.1331 - val_acc: 0.9900
Epoch 8/10
25/25 [==============================] - 14s 437ms/step - 

In [ ]:
print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['val_loss'])
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


with open('history3.txt', 'w') as f:
    f.write("acc")
    f.write('\n')
    f.write(str(history.history['acc']))    
    f.write('\n')
    f.write("val_acc")
    f.write('\n')
    f.write(str(history.history['val_acc']))
    f.write('\n')
    f.write("val_loss")
    f.write('\n')
    f.write(str(history.history['val_loss']))
    f.write('\n')
    f.write("loss")
    f.write('\n')  
    f.write(str(history.history['loss']))
    f.write('\n')      


## 5. Evaluation of model on the test dataset

In [ ]:
test_ds = keras.preprocessing.image_dataset_from_directory(
  'gdrive/MyDrive/2A/MODAL/testset4', labels='inferred', seed = 1, batch_size = 1, shuffle = True, image_size=(640, 480),color_mode='grayscale')

loss, acc = model.evaluate(test_ds)  # returns loss and metrics
print("loss: %.2f" % loss)
print("acc: %.2f" % acc)


Found 400 files belonging to 2 classes.
400/400 [==============================] - 4s 9ms/step - loss: 0.1026 - acc: 0.9950
loss: 0.10
acc: 1.00


In [ ]:
#confusion matrix

predictions = []
labels =  []
for x, y in test_ds:
  y_pred = model.predict(x)
  if y_pred[0][0]>0.5:
      predictions.append(0)
  else:
      predictions.append(1)

  labels.append(y.numpy()[0])


In [ ]:
confusion_mtx = tf.math.confusion_matrix(labels=np.array(labels), predictions=np.array(predictions)).numpy()
print(confusion_mtx)

#The matrix columns represent the prediction labels and the rows represent the real labels. 
#[0][0] -> true positive (circle classified as circle)
#[0][1] -> false negative (circle classified as square)
#[1][0] -> false positive (square classified as circle)
#[1][1] -> true negative (square classified as square)

[[200   0]
 [  2 198]]


# Raspberry dataset

In [ ]:
raspberry_ds = keras.preprocessing.image_dataset_from_directory('gdrive/MyDrive/2A/MODAL/raspberrytest', labels='inferred', batch_size = 1, image_size=(640, 480),color_mode='grayscale')

loss, acc = model.evaluate(raspberry_ds)  # returns loss and metrics
print("loss: %.2f" % loss)
print("acc: %.2f" % acc)

Found 4 files belonging to 2 classes.
4/4 [==============================] - 0s 6ms/step - loss: 1.1488 - acc: 0.7500
loss: 1.15
acc: 0.75


In [ ]:
rb_predictions = []
rb_labels =  []
i=0
for x, y in raspberry_ds:
  rb_y_pred = model.predict(x)
  if rb_y_pred[0][0]>0.5:
      rb_predictions.append(0)
      print("i = " + str(i))
      print("y = " + str(y.numpy()[0]))
  else:
      rb_predictions.append(1)
  i+=1

  rb_labels.append(y.numpy()[0])

rb_confusion_mtx = tf.math.confusion_matrix(labels=np.array(rb_labels), predictions=np.array(rb_predictions)).numpy()
print(rb_confusion_mtx)

i = 0
y = 0
i = 1
y = 0
i = 4
y = 0
i = 7
y = 0
[[4 2]
 [0 4]]


## Miscellaneous

In [ ]:
# # myfile = open('gdrive/MyDrive/2A/MODAL/data/train_cercle/attempt_4_circle_201_r_3.47_bruit_1.png')
# # plt.imshow(myfile)

# img = mpimg.imread('gdrive/MyDrive/2A/MODAL/data/train_cercle/attempt_4_circle_201_r_3.47_bruit_1.png')
# #img = tf.keras.utils.normalize(img,axis = 1)
# imgplot = plt.imshow(img)
# plt.show()
